In [119]:
import os
import json
import pandas as pd
import requests

import pyspark
from pyspark.sql.types import *
from pyspark.sql import SparkSession, Row

ss = SparkSession.builder.master("local[4]") \
                    .appName('CreateDataset') \
                    .getOrCreate()


# Helper functions

In [114]:
def get_audio_features_spotifyApi(track_uri):
    keys_to_remove = ['key','mode','type','track_href','analysis_url','time_signature','id', 'duration_ms']

    track_features = sp.audio_features(track_uri)
    track_features = track_features[0]
    print(track_features)
    for key in keys_to_remove:
        del track_features[key]
        
    popularity = sp.track(track_uri)['popularity']
    track_features['popularity'] = popularity       
    return track_features


In [9]:
PATH = '/home/afeka/Desktop/spotify_dataset/mpd.slice.13000-13999.json'

In [98]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
SPOTIPY_CLIENT_ID='ff464bf1719f4878a4c6b078329878fb'
SPOTIPY_CLIENT_SECRET='ccd5c137ece94d78b801a283b50f0340'
auth_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [99]:
DF = pd.DataFrame()
data = json.load(open(PATH))
DF = pd.concat([DF,pd.DataFrame.from_dict(data['playlists'])])

In [61]:
DF.drop([column for column in DF.columns if column != 'tracks'], axis=1, inplace=True)

In [62]:
DF

,tracks
0,"[{'pos': 0, 'artist_name': 'Bob Marley & The W..."
1,"[{'pos': 0, 'artist_name': 'Alice Boman', 'tra..."
2,"[{'pos': 0, 'artist_name': 'Sleeping With Sire..."
3,"[{'pos': 0, 'artist_name': 'Unk', 'track_uri':..."
4,"[{'pos': 0, 'artist_name': 'Sean Kingston', 't..."
...,...
995,"[{'pos': 0, 'artist_name': 'Chris Tomlin', 'tr..."
996,"[{'pos': 0, 'artist_name': 'John Mayer', 'trac..."
997,"[{'pos': 0, 'artist_name': 'The All-American R..."
998,"[{'pos': 0, 'artist_name': 'Bon Iver', 'track_..."


The tracks information stored in dict

In [63]:
DF.iloc[0][0][0]

{'pos': 0,
 'artist_name': 'Bob Marley & The Wailers',
 'track_uri': 'spotify:track:1QQfbfMfyzNZhT5fg3RpuS',
 'artist_uri': 'spotify:artist:2QsynagSdAqZj3U9HgDzjD',
 'track_name': 'Buffalo Soldier',
 'album_uri': 'spotify:album:1sXOM4wTy7TbTPdPz68YeF',
 'duration_ms': 257440,
 'album_name': 'Confrontation'}

Extract relevant track information

In [64]:
#Extract tracks from list
tracks_list = []
for tracks in DF['tracks']:
     for track in tracks:
        del track['pos'], track['album_uri'], track['duration_ms'], track['album_name']
        tracks_list.append(track)

In [65]:
tracks_list[0]

{'artist_name': 'Bob Marley & The Wailers',
 'track_uri': 'spotify:track:1QQfbfMfyzNZhT5fg3RpuS',
 'artist_uri': 'spotify:artist:2QsynagSdAqZj3U9HgDzjD',
 'track_name': 'Buffalo Soldier'}

The number of songs that we are dealing with

In [66]:
len(tracks_list)

66127

# Convert to Spark Dataframe

In [130]:
tracks_RDD = ss.sparkContext.parallelize(tracks_list)

Number of partitions we work

In [131]:
tracks_RDD.persist()
tracks_RDD.getNumPartitions()

4

In [132]:
columns= ["artist_name", "track_uri", "artist_uri","track_name"]
track_feature_SDF = ss.createDataFrame(data = track_features, schema = columns)

In [136]:
track_feature_SDF = track_feature_SDF.distinct()

In [138]:
tracks_list = track_feature_SDF.collect()

In [ ]:
track_info = []
for track in tracks_list:
    audio_features = get_audio_features_spotifyApi(track['track_uri'].split(':')[2])
    audio_features['track_name'] = track['track_name']
    audio_features['artist_name'] = track['artist_name']
    if audio_features is not None:
        track_info.append(audio_features)
tracks_df = pd.DataFrame(track_info)
tracks_df_without_dup = track_df.drop_duplicates()
tracks_df_without_dup.to_csv("spotify_dataset1.csv")

In [15]:
tracks_df.drop_duplicates().to_csv("spotify_dataset2.csv")